# Homework 3 - What is the best anime in the world?

Algorithmic Methods of Data Mining; Group #28

mohammadmehdirazavi97@gmail.com、zhangxufeng1998@gmail.com pristera.1657095@studenti.uniroma1.it

## Question 1: Data Collection


In [105]:
# from ADMhw3functions import *
# import modules
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import time
import os
import random
import datetime
import re
import heapq
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import math
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mehdi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 1.1 Get the list of animes

In [ ]:
test = 'https://myanimelist.net/anime/'
linkhead = 'https://myanimelist.net/topanime.php?limit='
anime_link = []
for i in range(0,19150,50):
    anime_link.append(linkhead+str(i)) #Generate the url where we need to get the anime url on it.

real_link = []
for link_item in anime_link:  
    link =getlink(link_item)
    while link == []:# Sometimes there is no info in the html we got (maybe for network or others). If it happens, this loop can correct it.
        link =getlink(link_item) # get the link
    time.sleep(0.5)
    for item in link:
        if (item != None) and (test in item) and ('video' not in item) and ('season?_location') not in item: #delete the links which we don't need
            real_link.append(item)
    
    real_link = real_link[0:-10] # delete the links at the bottom of the web page. Thera are animes, but they also appear in the top anime list.

In [ ]:
link_set = []
for i in range(len(real_link)):
    if i%2 == 0:
        link_set.append(real_link[i])
# What we get have two same items for each anime before. So we wrote a loop to delete the duplicates

In [ ]:
with open('link_set.txt','w',encoding = 'utf-8') as f:
    for i in link_set:
        f.write(i+'\n')

f.close # Finally, we saved links at local directory.

### 1.2 Crawl animes

In [ ]:
f = open('/content/drive/MyDrive/Algorithmic methods for data mining/HW3/Data/link_set.txt','r',encoding = 'utf-8') 
link_set = f.readlines()
link = []
for line in link_set:
    link.append(line)
# load the link data and transfer it to list
print(pd.Series(link).head())
print("Total number of the urls is "+str(len(link)))

0    https://myanimelist.net/anime/5114/Fullmetal_A...
1       https://myanimelist.net/anime/28977/Gintama°\n
2    https://myanimelist.net/anime/38524/Shingeki_n...
3     https://myanimelist.net/anime/9253/Steins_Gate\n
4    https://myanimelist.net/anime/42938/Fruits_Bas...
dtype: object
Total number of the urls is 19131


In [ ]:
for i in range(0,383): #You can change the number here for part data set; you need to replace it by your data range. 
    path = 'C:\\Users\\zhang\\Desktop\\Untitled Folder\\page '+ str(i+1) # path we store the html
    os.mkdir(path) # For each page, we creat a folder
    os.chdir(path) 
    for j in range(i*50,(i+1)*50): # count the number of files in each folder.
        new_url = ''
        aurl = link[j]
        sums = 0
        for k in aurl: # Beacuse there are some ascii characters in url, we delete the name of animes in urls. It still works after we delete the name. 
            if k == '/':
                sums = sums +1
            if sums != 5:
                new_url = new_url + k

        ahtml = getHtml(new_url)
        saveHtml('Anime '+str(j+1), ahtml)

    print('Page '+str(i+1)+' downloaded successfully.') 

### 1.3 Parse downloaded pages

In [ ]:
# File organize; Put all htmls into one folder.
import os
import shutil

#source_path = os.path.abspath(r'/content/drive/MyDrive/Algorithmic methods for data mining/HW3/HTML/Mehdi HTML')
source_path = os.path.abspath(r'/content/drive/MyDrive/Algorithmic methods for data mining/HW3/HTML/Xufeng HTML')
#source_path = os.path.abspath(r'/content/drive/MyDrive/Algorithmic methods for data mining/HW3/HTML/Daniele HTML/File 141-280')
target_path = os.path.abspath(r'/content/drive/MyDrive/Algorithmic methods for data mining/HW3/All html')

if not os.path.exists(target_path):
    os.makedirs(target_path)

if os.path.exists(source_path):
    for root, dirs, files in os.walk(source_path):
        for file in files:
            src_file = os.path.join(root, file)
            shutil.copy(src_file, target_path)
            print(src_file)

print('copy files finished!')


In [ ]:
import glob
path_file_number=glob.glob('/content/drive/MyDrive/Algorithmic methods for data mining/tsv_HW3/*.html') 
print(path_file_number)
print(len(path_file_number))
# count for the number of files in specific path

In [ ]:
file_name = "Anime "
file_name_list = []
for i in range(0,19131):
    file_name_list.append(file_name + str(i+1)+'.html')
file_path = []
for j in range(0,19131):
    file_path.append('/content/drive/MyDrive/Algorithmic methods for data mining/HW3/All html/'+str(file_name_list[j]))
# creat the path list thst we stored the htmls

In [ ]:
tsv_path = '/content/drive/MyDrive/Algorithmic methods for data mining/tsv_HW3/anime_'
tsv_name = []
for k in range(0,19131):
  tsv_name.append(str(k+1)+'.tsv')
tsv_path_list = []
for l in range(0,19131):
  tsv_path_list.append(tsv_path+tsv_name[l])
# creat the path list that we store tsv file for each anime

In [ ]:
for m in range(0,19131):
  with open(tsv_path_list[m],'w',encoding = 'utf-8') as f:
      tsv = ' \t '
      table_head = 'animeTitle'+tsv+'animeType'+tsv+'animeNumEpisode'+tsv+'releaseDate'+tsv+'endDate'+tsv+'animeNumMembers'+tsv+'animeScore'+tsv+'animeUsers'+tsv+'animeRank'+tsv+'animePopularity'+tsv+'animeDescription'+tsv+'animeRelated'+tsv+'animeCharacters'+tsv+'animeVoices'+tsv+'animeStaff'+'\n'
      f.write(table_head)
      temp = getinfo(file_path[m])# get the info by using functions we defined
      temp = temp + '\n'
      f.write(temp)
  f.close
  print('file ' + str(m) +' finished.')
  # get info from html and write it into tsv file

In [ ]:
# put all tsv files into one file
with open('/content/drive/MyDrive/Algorithmic methods for data mining/HW3/all_info_tsv/animes.tsv','w',encoding = 'utf-8') as allf:
  for n in range(0,19131):
    with open(tsv_path_list[n],'r+',encoding = 'utf-8') as sf:
      for o in range(2):
        lineinfo = sf.readline()
        if n == 0 and o == 0:
          allf.write(lineinfo)
        if o == 1:
          allf.write(lineinfo)
    sf.close
allf.close

In [9]:
# Describe the data
import pandas as pd
anime_data = pd.read_csv('animes.tsv',sep = '\t')
print(anime_data.head())
print("There are "+ str(len(anime_data)) + ' cases in this DataFrame.')

                          animeTitle   animeType   animeNumEpisode   \
0  Fullmetal Alchemist: Brotherho...          TV                64    
1                           Gintama°          TV                51    
2  Shingeki no Kyojin Season 3 Pa...          TV                10    
3                        Steins;Gate          TV                24    
4           Fruits Basket: The Final          TV                13    

   releaseDate       endDate    animeNumMembers   animeScore   animeUsers   \
0   2009-04-05    2010-07-04             2676862        9.16      1629051    
1   2015-04-08    2016-03-30              484040        9.09       169476    
2   2019-04-29    2019-07-01             1597175        9.09      1087519    
3   2011-04-06    2011-09-14             2091647        9.09      1109700    
4   2021-04-06    2021-06-29              275549        9.07       113310    

   animeRank    animePopularity   \
0          1                   3   
1          2                 337

## Question 2: Search Engine


### 2.1 Conjunctive query

In [10]:
df = pd.read_csv('animes.tsv',sep = '\t')
data_array = np.array(df)
anime_name = data_array[0:,0]
anime_des = data_array[0:,10]

In [101]:
# process the description with NLTK


# get a set with all words that appear in description.
all_words = []
for k in range(19131):
  all_words = all_words + des_words[k]

# get a set with all words (dict keys) 
all_words = set()
for k in range(19131):
  all_words = all_words.union(set(des_words[k]))

# maps each word to an integer
map_words = pd.Series(list(all_words))
map_words.to_csv('/content/drive/MyDrive/Algorithmic methods for data mining/HW3/words_set.csv')



In [ ]:
#create the blank index
all_words = list(all_words)
index_dict = {}
for l in range(len(all_words)):
  index_dict[l]=[]

# fill in the blank index with doucument_id
for seq in range(19131):
  for word in des_words[seq]:
    Num = all_words.index(word)
    index_dict[Num].append(seq)

# Save the index
np.save('/content/drive/MyDrive/Algorithmic methods for data mining/HW3/doucment_index.npy', index_dict) 


In [102]:
# load local data. Most data I have loaded before.
read_dictionary = np.load('document_index.npy',allow_pickle = True).item() 
all_words = pd.read_csv('words_set.csv')
all_words = list(all_words['0'])
df = pd.read_csv('animes.tsv',sep = '\t')
data_array = np.array(df)
anime_name = data_array[0:,0]
anime_des = data_array[0:,10]
f = open('link_set.txt','r',encoding = 'utf-8') 
link_set = f.readlines()
link = []
for line in link_set:
  line = line[0:-1]
  link.append(line)

Examples of first search engine:

[Code of Search Engine function is in *ADMhw3functions.py*]

In [ ]:
# You can find the functions we defined in ADMhw3functions.py
output1 = SearchEngine() 
output1

Please input the key words:
dragon ball


,animeTitle,animeDescription,Url
1,Dragon Ball Movie 3: Makafushi...,Emperor Chiaotzu&#039;s wife has gone missing...,https://myanimelist.net/anime/892/Dragon_Ball_...
2,Dragon Ball,Gokuu Son is a young boy who lives in the woo...,https://myanimelist.net/anime/223/Dragon_Ball


In [ ]:
output2 = SearchEngine()
output2

Please input the key words:
help friend


,animeTitle,animeDescription,Url
1,Karneval (TV),"While in search of his precious friend, a you...",https://myanimelist.net/anime/16035/Karneval_TV
2,Clannad,Tomoya Okazaki is a delinquent who finds life...,https://myanimelist.net/anime/2167/Clannad
3,Boruto: Naruto the Movie,"The spirited Boruto Uzumaki, son of Seventh H...",https://myanimelist.net/anime/28755/Boruto__Na...
4,Hibike! Euphonium Movie 1: Kit...,After swearing off music due to an incident a...,https://myanimelist.net/anime/31989/Hibike_Eup...
5,Little Monica Monogatari,"Will, an orphan originally from Little Monica...",https://myanimelist.net/anime/1785/Little_Moni...
...,...,...,...
117,Crayon Shin-chan,There is no such thing as an uneventful day i...,https://myanimelist.net/anime/966/Crayon_Shin-...
118,Nanatsuko Nezumi Series,Seven mouse children and their normal lives. ...,https://myanimelist.net/anime/35619/Nanatsuko_...
119,Eun-sil-i,"In-hye and Sun-mi visit their hometown, a sma...",https://myanimelist.net/anime/30193/Eun-sil-i
120,Hashiri Tsuzukete Yokattatte.,Minato visits a recitation at an anime event ...,https://myanimelist.net/anime/38254/Hashiri_Ts...


### 2.2 Conjunctive query & Ranking score


In [ ]:
# Calculate the tfidf, and reorganize the dictionary
for i in range(len(all_words)):
  tfidf_list = tfidf(i)
  key_num = read_dictionary[i]
  new_key_value = []
  for j in range(len(key_num)):
    newitemij = [tfidf_list[j],key_num[j]]
    new_key_value.append(newitemij)
  read_dictionary[i] = new_key_value

In [ ]:
for i in range(5):  
  print(read_dictionary[i])
# Now we get a dictionary index with tfidf and document_id  ---->  index{i}:[tfidf,document_id]

[[0.06401990477261936, 1763]]
[[0.09761450826716218, 16789]]
[[0.050559309410171185, 18290]]
[[0.06636706853269145, 4361], [0.04840029307356503, 6017], [0.08505294219723564, 19112]]
[[0.0510832400776341, 1533]]


Examples of SearchEngine2:

[Code of SearchEngine2 function is in *ADMhw3functions.py*]

In [ ]:
print("Please input the key words:")
key_words = input().split()
output3 = SearchEngine2(key_words)
output3

Please input the key words:
dragon ball


,animeTitle,animeDescription,Url,Similarity
1,Dragon Ball Movie 3: Makafushi...,Emperor Chiaotzu&#039;s wife has gone missing...,https://myanimelist.net/anime/892/Dragon_Ball_...,0.212814
2,Dragon Ball,Gokuu Son is a young boy who lives in the woo...,https://myanimelist.net/anime/223/Dragon_Ball,0.079556


In [ ]:
print("Please input the key words:")
key_words = input().split()
output4 = SearchEngine2(key_words)
output4

Please input the key words:
help friend


,animeTitle,animeDescription,Url,Similarity
1,Shi Ye Miao Yin,Yang Yao is a renowned exorcist who drives ou...,https://myanimelist.net/anime/44411/Shi_Ye_Mia...,0.206284
2,Esper Mami,Mami Sakura used to be a normal junior high s...,https://myanimelist.net/anime/3023/Esper_Mami,0.202260
3,Dengeki Oshioki Musume Gootama...,Mari Amachi is a very devout Christian who tr...,https://myanimelist.net/anime/5025/Dengeki_Osh...,0.199007
4,Tayo-ui Jangnangam Adventure,When Bella finds out that all the toys were n...,https://myanimelist.net/anime/40030/Tayo-ui_Ja...,0.196960
5,Bakemonogatari,"Koyomi Araragi, a third-year high school stud...",https://myanimelist.net/anime/5081/Bakemonogatari,0.192308
6,Lulin Da Maoxian,When a girl named Rainie runs away from home ...,https://myanimelist.net/anime/31205/Lulin_Da_M...,0.189737
7,Hidari no O&#039;Clock!!,Yuu Shinozaki is fed up with living according...,https://myanimelist.net/anime/8967/Hidari_no_O...,0.178647
8,Tsumamigui,What can a widowed woman do when she gets tha...,https://myanimelist.net/anime/1403/Tsumamigui,0.178017
9,Xiong Chumo: Tanxian Riji,Ninth season of Boonie Bears. The Bear Brothe...,https://myanimelist.net/anime/37102/Xiong_Chum...,0.174964
10,Kodai Ouja Kyouryuu King,Dr. Taylor was on a paleontology mission with...,https://myanimelist.net/anime/3576/Kodai_Ouja_...,0.173749


## Question 3: Define a new score!


In [122]:
normInt = lambda x : [int(truncate(str(i).replace(',',''))) if i == i else 0 for i in x]
normString = lambda x : [i.translate(str.maketrans('', '', string.punctuation)).lower().split() if len(i) > 0 else None for i in x.fillna('')  ]
normFloat = lambda x : [round(float(str(i).replace(',',''))) if i == i else 0 for i in x]
newdf = pd.DataFrame()
newdf['animeTitle'] = normString(df['animeTitle '])
newdf['animeTitle1'] = df['animeTitle ']
newdf['animeType'] = normString(df[' animeType '])
newdf['animeNumEpisode'] = normString(df[' animeNumEpisode '])
newdf['releaseDate'] = normDate(df[' releaseDate '])
newdf['endDate'] = normDate(df[' endDate '])
newdf['animeNumMembers'] = np.int64(df[' animeNumMembers '])
newdf['animeScore'] = convert_dtype_float(df[' animeScore '])
newdf['animeUsers'] = normString(df[' animeUsers '])
newdf['animeRank'] = normString(df[' animeRank '])
newdf['animePopularity'] = np.int64(df[' animePopularity '])
newdf['animeDescription'] = normString(df[' animeDescription '])
newdf['animeRelated'] = normString(df[' animeRelated '])
newdf['animeCharacters'] = normString(df[' animeCharacters '])
newdf['animeVoices'] = normString(df[' animeVoices '])
newdf['animeStaff'] = normString(df[' animeStaff'])
newdf['index'] = np.arange(1,19132,1)
df['index'] = np.arange(1,19132,1)


After receiving an input from a user, we can assign to each word and each number in the query an update score. Such score depends on a wide list of rules: let's look at all those rules in details:

For each element $w_i$ in the query:
  - if there is a match between the query-element and an element belonging to the ***'animeTitle'*** variable:            $score+= (1/len(t))$***2**       
  (where ***'t'*** is the animeTitle that contains the matched element, and ***'2'*** is the 'weight' of the match)
 
  - if there is a match between the query-element and an element belonging to ***'animeStaff'*** variable:            $score+= (1/len(s))$***1.5**     
  (where ***'s'*** is the animeStaff that contains the matched element, and ***'1.5'*** is the 'weight' of the match)
  
  
  - if there is a match between the query-element and an element belonging to ***'animeCharacters'*** variable:            $score+= (1/len(c))$***1.5**       
  (where ***'c'*** represents the animeCharacters that contain the matched element, and ***'1.5'*** is the 'weight' of the match)
  
  - if there is a match between the query-element and an element belonging to ***'animeVoices'*** variable:            $score+= (1/len(v))$***1.5**       
  (where ***'v'*** represents the animeVoices that contain the matched element, and ***'1.5'*** is the 'weight' of the match)
  
  
  - if there is a match between the query-element and an element belonging to ***'animeType'*** variable:            $score+= $***0.5***       
  (where ***'0.5'*** is the 'weight' of the match)
  
  
  
  - if there is a match between the query-element and an element belonging to ***'animeScore'*** variable:            $score+= $***0.5***       
  (where ***'0.5'*** is the 'weight' of the match)
  
  
  - if there is a match between the query-element and an element belonging to ***'animeNumEpisode'*** variable:            $score+= $***0.75***       
  (where ***'0.75'*** is the 'weight' of the match)
  
  
  - if there is a match between the query-element and an element belonging to ***'animePopularity'*** variable:            $score+= $***0.5***       
  (where ***'0.5'*** is the 'weight' of the match)


$score=\sum_{i\in index}\sum_{q\in query}2\frac{1[q\in animeTitle_i]}{\left |animeTitle_i\right |}+1.5\frac{1[q\in animeStaff_i]}{\left |animeStaff_i\right |}+1.5\frac{1[q\in animeCharacters_i]}{\left |animeCharacters_i\right |}+1.5\frac{1[q\in animeVoices_i]}{\left |animeVoices_i\right |}+1\frac{0.5[q\in animeType_i]}{\left |animeType_i\right |}+\frac{0.5[q\in animeScore_i]}{\left |animeScore_i\right |}+\frac{0.75[q\in animeNumEpisode_i]}{\left |animeNumEpisode_i\right |} + \frac{0.5[q\in animePopularity_i]}{\left |animePopularity_i\right |} + 1cosine\:similarity_{i,q}$

Example of new scoring function:

[Code of search function is in *ADMhw3functions.py*]

In [137]:
# Code of search function is in *ADMhw3functions.py*

key_words = input()
result = search(key_words)
result


,animeTitle,animeDescription,Url,score
2,Dragon Ball,Gokuu Son is a young boy who lives in the woo...,https://myanimelist.net/anime/223/Dragon_Ball,2.079556
1,Dragon Ball Movie 3: Makafushi...,Emperor Chiaotzu&#039;s wife has gone missing...,https://myanimelist.net/anime/892/Dragon_Ball_...,1.012814


## Question 4: Understanding the anime's reviews


We previously parsed the html pages for the first 4 comments of each anime.
In each review we saved the first 500 characters.
We stored the reviews in 'review.tsv' file.

In [164]:
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from nltk.sentiment import SentimentIntensityAnalyzer

review = pd.read_csv('./review.tsv',sep = '\t')

Below we used clean_text function in order to clean the reviews string.

In [144]:
nlp = spacy.load('en_core_web_sm')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def clean_text(text):
    words = word_tokenize(text)
    filtered_words = []
    for word in words:
        if word.lower() not in stop_words and word.isalpha():
            filtered_words.append(lemmatizer.lemmatize(word.lower()))
    return filtered_words

In [151]:
word_list = review.iloc[:, 1].apply(lambda text: clean_text(text))
# add a list of words for each anime
df['review_words'] = word_list

Here you can see the top 20 words with most accurance.

In [176]:
flat_list = [item for sublist in word_list for item in sublist]
pd.Series(flat_list).value_counts()[:20]

anime        25089
story        21752
quot         19681
one          16198
character    16143
like         14642
series       14184
show         13384
episode      12846
first        11335
br           10612
really        9910
good          8950
time          8856
movie         7841
get           7616
season        7406
much          7346
girl          6818
thing         6584
dtype: int64

In [175]:
# Review compounds score using SentimentIntensityAnalyzer model

sia = SentimentIntensityAnalyzer()
review_compound = []
for item in word_list:
    text =''
    for word in item:
        text += word + ' '
    if sia.polarity_scores(text)["compound"] > 0:
        review_compound.append('pos')
    else:
        review_compound.append('neg')
df['review_compound'] = review_compound

In [218]:
df.groupby([' animeType ', 'review_compound']).count()[[ 'animeTitle ']]

animeTitle 
 animeType  review_compound             
            neg                     1459
            pos                      240
 Movie      neg                     2163
            pos                     1314
 ONA        neg                     1580
            pos                      630
 OVA        neg                     2048
            pos                     1885
 Special    neg                     1470
            pos                      823
 TV         neg                     2393
            pos                     3126

As you can see anime with TV type has the most positive compounds. After That Movie type has the second most positive compounds

In [193]:
from scipy.stats import spearmanr
corr , _ = spearmanr(df[' animeNumEpisode '], df['review_compound'])
corr

0.08762809044942291

With the result above we can see that there is not any relation between number od episodes and the review compound

## Question 5: Algorithmic question

In [1]:
import time
import numpy as np
np.random.seed(50)  # seed for reproducibility
n=20000 # size of array
Min=1 # min number
Max=10 # max number
appointments = np.random.randint(Min,Max, size=n)  # One-dimensional array with casual integer number betweenmin and max
appointments=appointments*10 #to have all of the elements as multiple of 10

In [2]:
#recursive function (almost correct)

def maximaizeschedule_2 (schedule, total):
    del_v = []
    n = len(schedule)
    check = np.zeros(n, dtype= int) 
    i=np.amax(schedule)
    
    for j in range(0, n ,1) : 
    
        if j == 0:               
                if check[j+1] != 1:
                    if schedule[j] == i:   
                        total+=i      
                        check[j] = 1   
                        del_v.extend((j,j+1))
                        
        if j == n-1:             
            if check[j-1] != 1:   
                if schedule[j] == i:
                    total+=i
                    check[j] = 1
                    del_v.extend ((j-1,j))

        if  j!=0  and   j != n-1: 
            if check[j+1] != 1 and check[j-1] != 1:  
                if schedule[j] == i:
                    total+=i
                    check[j] = 1
                    del_v.extend ((j-1, j ,j+1))
        
    del_v = set(del_v)
    a = list(schedule)
    
    for index in sorted(del_v, reverse=True):
        del a[index]   
    if len(a) == 0:
        return total
    
    if len(a) == 1 :
        return (total + a[0])

    else:
        return maximaizeschedule_2(np.array(a), total)

In [3]:
#function with for loops

def maximaizeschedule_1 (schedule): 
    n = len(schedule)
    check = np.zeros(n, dtype= int) #array of zeros used to check positions
    total = 0  #total value initializaion

    for i in range(np.amax(schedule), 0 , -10): # range of possible value
        for j in range(0, n ,1) :   # for each possible value the whole array is searched

            if j == 0:               # boundaries condition (starting point)
                if check[j+1] != 1:
                    if schedule[j] == i:   # if at first position you only need to check the following
                        total+=i      # if found and condition si fulfilled sum that value to total
                        check[j] = 1    #assign the position of a taken value to not repeatedly take it

            elif j == n-1:             # boundaries condition (ending point)
                if check[j-1] != 1:   #if at last position you only need to check the prevous
                    if schedule[j] == i:
                        total+=i
                        check[j] = 1

            else:
                if check[j+1] != 1 and check[j-1] != 1:  #checking if the value is permitted to be take
                    if schedule[j] == i:
                        total+=i
                        check[j] = 1
    return total         

In [4]:
start_time = time.time()
result=maximaizeschedule_1(appointments)
print("The maximum time that the personal trainer could work is:", result)
print("The execution time of the alghoritm is:  %s seconds" % (time.time() - start_time))

The maximum time that the personal trainer could work is: 551410
The execution time of the alghoritm is:  0.07366204261779785 seconds


In [ ]:
start_time = time.time()
result=maximaizeschedule_2(appointments, 0)
print("The maximum time that the personal trainer could work is:", result)
print("The execution time of the alghoritm is:  %s seconds" % (time.time() - start_time))

The maximum time that the personal trainer could work is: 497490
The execution time of the alghoritm is:  0.19008183479309082 seconds


Below you can see the time analysis of the algorithm

Time Analysis <img src="img.jpg" width=600 height=800 />


<img src="img1.jpg" width=800 height=600 />
